<a href="https://colab.research.google.com/github/c-etulle/Vegetation-dynamics-in-the-Subandean-grasslands-of-Chubut/blob/main/2_Import_MODIS_NDVI_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Final output:** Die monatlichen NDVI-time-series (2000 - 2024) für alle MARAS-sample-locations werden separat in Chubut als csv gespeichert.

# 1.) Import packages & libraries

In [ ]:
#!pip install -q geopandas # geopanda is Python package for working with geospatial data ; q for quiet (usual logs not shown)
%pip install unidecode # library that transforms special characters

from unidecode import unidecode
import geemap # for making the maps
import pandas as pd # provide data structures like series and dfs
import geopandas as gpd
import json
import ee # imports Earth Engine
import math
import numpy as np

ee.Authenticate() # pop-up asking to log into google account to & grant permission to GEE
ee.Initialize(project='ee-meinzinger-patagonia')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 9.8 MB/s eta 0:00:00


# 2.) Create access to Google Drive & set working directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # dt. einbinden
#change working directory:
%cd /content/drive/My\ Drive/ee-meinzinger-patagonia/Data

Mounted at /content/drive
/content/drive/My Drive/ee-meinzinger-patagonia/Data


In [ ]:
!pwd # test: print working directory
!ls -al # test: getting info about my current Colab environment

/content/drive/My Drive/ee-meinzinger-patagonia/Data
total 2729
-rw------- 1 root root       5 Aug  2 10:20 Chubut_Mainland.cpg
-rw------- 1 root root     687 Aug  2 10:20 Chubut_Mainland.dbf
-rw------- 1 root root     145 Aug  2 10:20 Chubut_Mainland.prj
-rw------- 1 root root     132 Aug  2 10:20 Chubut_Mainland.sbn
-rw------- 1 root root     116 Aug  2 10:20 Chubut_Mainland.sbx
-rw------- 1 root root  925644 Aug  2 10:20 Chubut_Mainland.shp
-rw------- 1 root root   13484 Aug  2 10:20 Chubut_Mainland.shp.xml
-rw------- 1 root root     108 Aug  2 10:20 Chubut_Mainland.shx
-rw------- 1 root root       5 Sep  7 13:09 Chubut_samples.CPG
-rw------- 1 root root 1785926 Sep  7 13:09 Chubut_samples.dbf
-rw------- 1 root root    8614 Sep  8 08:05 Chubut_samples_points_processed.csv
-rw------- 1 root root     145 Sep  7 13:09 Chubut_samples.prj
-rw------- 1 root root    2932 Sep  7 13:09 Chubut_samples.sbn
-rw------- 1 root root     340 Sep  7 13:09 Chubut_samples.sbx
-rw------- 1 root root   

# 3.) Define all functions

**OBS.** **Landsat mission 4 & 5:** NIR (band 4) and red (band 3).
**Landsat mission 8 & 9:** NIR (band 5) and red (band 4). Note: I probably do not need this code chunk, since the NDVI in MODIS is "already calculated".

In [ ]:
# Applying scaling factors for Landsat images = converting their DNs to physically meaningful units (e.g., reflectance for optical bands)
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2) # selects all bands that start with SR_B
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands).addBands(thermalBands)

# Cloud masking function for Landsat images ; I basically get an image free of cloud shadow, cloud and snow.
def maskLsr(image):
  watersBitMask = ee.Number(2).pow(7).int()
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  snowsBitMask = ee.Number(2).pow(4).int()
  qa = image.select('QA_PIXEL') # qa = quality assessment
  mask = (qa.bitwiseAnd(cloudShadowBitMask).eq(0)
    .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    .And(qa.bitwiseAnd(snowsBitMask).eq(0))
    .And(qa.bitwiseAnd(watersBitMask).eq(0)))
  return image.updateMask(mask)

# Defining bands for Landsat 5 & 7: NIR (4) & Red (3) ; SR_B4 = B4 (NIR) & SR_B3 = B3 (Red)
def renameBand_L5_7(img):
  return img.select(['SR_B2','SR_B3','SR_B4','SR_B5'],['B2', 'B3', 'B4', 'B5'])

# Defining / Renaming bands for Landsat 8 & 9: NIR (5) & Red (4) ; SR_B5 = B4 (NIR) & SR_B4 = B3 (Red)
def renameBand_L8_9(img):
  return img.select(['SR_B2','SR_B3','SR_B4','SR_B5'],['B5', 'B2', 'B3', 'B4'])

# Calculates NDVI
def calcNDVI(image):
  ndvi = image.normalizedDifference(['B4', 'B3']).multiply(10000).toInt().rename('NDVI')
  return image.addBands(ndvi)

# 4.) Get coordinates for sample locations & roi

### a.) Modify Chubut_samples.shp

In [ ]:
path_points = r'Chubut_samples.shp'

# Shapefile is read into a GeoDf:
points = gpd.read_file(path_points)

# Select only the relevant columns for myscript ; geometry = coordinates
points = points[["Country", "Province", "Site_Name", "ID", "geometry"]]

# Remove duplicated rows ; duplicates identified considering "Site_Name, "Country" and "Province"
points = points \
.drop_duplicates(subset=["Site_Name", "Country", "Province"], keep="first") \
.reset_index(drop = True)

# Get the site_list:
site_list = (points['Site_Name']) # extracts site names from Chubut sample points

# Change special characters:
site_list = site_list.str.replace('[()°]', '', regex=True) # removes specifically ()°
site_list = [unidecode(string) for string in site_list]
site_list = pd.Series(site_list) # converts back to pd.Series object, because the original site_list is a pd.Series object

# Add these modified sites back to the "points" GeoDf:
points["Site_Name"] = site_list

# Save as .csv to check it:
points_save = points.copy() # important to use .copy() ; otherwise, instead of making a new point file, the original "points" would be changed whenever "points_save" is changed
points_save['geometry'] = points_save['geometry'].apply(lambda x: x.wkt) # convert geometry into wkt, which csv can handle
points_save.to_csv('Chubut_samples_points_processed.csv', index=False)

# For testing, limit the size of the dataset
# points = points[0:5]
# site_list = site_list[0:5]

# test to check that this time site_list and GeodDf are identical:
print(points.head)
print(site_list.head)

<bound method NDFrame.head of        Country Province      Site_Name       ID                     geometry
0    Argentina   Chubut           CERM  #CH-001  POINT (-70.30031 -45.44061)
1    Argentina   Chubut         La Ana  #CH-002  POINT (-70.26778 -45.42675)
2    Argentina   Chubut     Media Luna  #CH-003  POINT (-71.39069 -45.58531)
3    Argentina   Chubut  Alto Rio Mayo  #CH-004  POINT (-71.31325 -45.56728)
4    Argentina   Chubut   1 San Felipe  #CH-005  POINT (-66.01511 -44.20461)
..         ...      ...            ...      ...                          ...
98   Argentina   Chubut   LAS MERCEDES  #CH-144  POINT (-70.92572 -44.69858)
99   Argentina   Chubut  Nueva Lubecka  #CH-145  POINT (-70.34400 -44.53789)
100  Argentina   Chubut        El Poyo  #CH-146  POINT (-70.74372 -43.39158)
101  Argentina   Chubut      El Saucal  #CH-147  POINT (-70.32483 -42.80264)
102  Argentina   Chubut           Lino  #CH-148  POINT (-71.00556 -42.03336)

[103 rows x 5 columns]>
<bound method NDFrame

<ipython-input-6-4f4b1b5baa0e>:27: UserWarning: Geometry column does not contain geometry.
  points_save['geometry'] = points_save['geometry'].apply(lambda x: x.wkt) # convert geometry into wkt, which csv can handle


### b.) Convert the points to json, then to eeFeatureCollection

Note: prist --> points_fc = collection of all the sample location's coordinates

In [ ]:
prist = json.loads(points.to_json()) # converts to json - now each row is converted into something that looks like a dictionary in one large dictionary
#print(prist) # each object within {}

# converts to feature collection ; here it requires to already have initialized the ee-project
# the ee featureCollection is yet another format of the same information as in the GeoDf above
points_fc = ee.FeatureCollection(prist)
# print(points_fc)

### c.) Make a list with all the coordinates

For plotting them with the ROI

In [ ]:
coord_list = [] # creates an empty list
for l in range(len(prist['features'])):
  coords_p = np.array(prist['features'][l]['geometry']['coordinates']) # gets the coordinates for each sample location
  coord_list.append(coords_p.tolist())

print(coord_list)

[[-70.3003060003216, -45.440611000397325], [-70.26777800035438, -45.42674999971268], [-71.39069400002802, -45.585306000071796], [-71.31324999988755, -45.5672779996799], [-66.01511100007258, -44.20461100009754], [-66.04972199987037, -44.1894440003403], [-66.39175000026228, -45.01930599987196], [-66.11863899979, -44.87386099978528], [-65.39138900000233, -43.60833300026769], [-65.57188899967753, -42.962055999684594], [-65.35052799976734, -44.295499999725166], [-64.5623889997525, -42.96672199992031], [-68.67308299998041, -43.82805600018423], [-68.5869169996198, -43.87049999967519], [-68.53991699956981, -43.942360999560435], [-69.00627800042935, -43.596417000244344], [-68.90058299970565, -43.56166700033174], [-68.20011100002262, -42.888360999760266], [-68.227028000242, -42.750110999622905], [-67.10249999997495, -42.46547199993279], [-66.11033299986804, -42.51005600008432], [-67.1682220000452, -44.97147200003275], [-67.09650000027477, -45.0759440001155], [-68.01663899998982, -44.729193999603

## 4.2) Region Of Interest (ROI): Get the coordinates of the border of Chubut Mainland

In [ ]:
path_admin = r'Chubut_Mainland.shp'
admin = gpd.read_file(path_admin)
g = json.loads(admin.to_json())
coords = np.array(g['features'][0]['geometry']['coordinates'])

roi = ee.Geometry.Polygon(coords.tolist())
# print(roi)

## 4.3) Visualize the ROI with sample sites

In [ ]:
m = geemap.Map() # make an empty map called "m"

# add the ROI:
m.add_layer(roi, {'color': 'white'}, 'Chubut')

# Make points as geometry object
multi_point = ee.Geometry.MultiPoint(coord_list)

# Add the multipoint layer to the map
m.addLayer(multi_point, {'color': 'blue'}, 'Sample sites')

m.center_object(roi, 6)
m # map is getting printed

Map(center=[-43.786686612294886, -68.51206582534277], controls=(WidgetControl(options=['position', 'transparen…

# 5.) Load MODIS images & get NDVI

## 5.1) Extract and calculate NDVI images, and save as csv

In [ ]:
# Load MOD13A1 dataset
modis = ee.ImageCollection('MODIS/006/MOD13A1').select('NDVI')

# Function to compute the mean NDVI
def getNDVI(image):
    meanDict = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=pt.geometry(),
        scale=500,  # MODIS NDVI has a 500m resolution
        maxPixels=1e13
    )
    mean = ee.Algorithms.If(meanDict.contains('NDVI'), meanDict.get('NDVI'), None)
    return ee.Feature(None, {'year': image.get('year'), 'month': image.get('month'), 'NDVI': mean})

# Define the MODIS dataset
modis = ee.ImageCollection('MODIS/006/MOD13A1').select('NDVI')

# Iterate over each site and process
for i, site in zip(range(points_fc.size().getInfo()), site_list):
    print(site)

    # Get the point feature
    pt = ee.Feature(points_fc.toList(points_fc.size().getInfo()).get(i))

    # Filter and process MODIS data
    monthlyMedianNDVI = (
        ee.ImageCollection.fromImages(
            ee.List.sequence(2000, 2024).map(
                lambda year: ee.List.sequence(1, 12).map(
                    lambda month: modis
                        .filterDate(ee.Date.fromYMD(year, month, 1), ee.Date.fromYMD(year, month, 1).advance(1, 'month'))
                        .median()
                        .set('year', year)
                        .set('month', month)
                )
            ).flatten()
        )
    )

    # Map the getNDVI function to extract statistics
    monthlyMedianNDVIStats = monthlyMedianNDVI.map(getNDVI)
    monthlyMedianNDVIFeatureCollection = ee.FeatureCollection(monthlyMedianNDVIStats)

    # Define the filename and export task
    filename = '{}_NDVI'.format(site)
    task = ee.batch.Export.table.toDrive(
        collection=monthlyMedianNDVIFeatureCollection,
        description=filename,
        folder='modis-2000-2024',
        fileFormat='CSV'
    )

    # Start the export task
    task.start()

CERM
La Ana
Media Luna
Alto Rio Mayo
1 San Felipe
2 San Felipe
Cerro Condor
La Isabel
La Clara
Las Piedritas
Berna
Bahia Cracker
San Sebastian
Bella Vista
La Regina
Chacra Berwyn
La Payanca
Los 5 Hermanos
Cerco de Piedras
Don Julian
El Moro
1 San Jose
2 San Jose
La Portena
La Juanita
El Castillo
Cerro Toro
El Rosillo
El Pajarito
La Maria
El Mollar
Valdes Creek Secc La Adela
Valdes Creek Secc. El Piquillin
La Travesia
La Concepcion
El Porvenir
La Esperanza Biedma
La Lonja
La Veneta
El Pato
La Sonia
El Oasis
Aguada Amarga
El Bosque
El Riscoso
La Esperanza Rio Senguer
1 Rio Guenguel
El Bajo
2 Rio Guenguel
Los Uncos
La Buena Suerte
Manantiales Behr
Numancia
Arroyo Chalia
Chali Aike
Canadon Blanco
La Cancha
Yague
El Maiten
Tureo
Seis Hermanos
La Anita
Gonzalez
El Guri
Los Guindos
Los Tanos
Los Luises
La Querencia
El Solito
Arabia Saudita
El Zaino
Maria Edith
La Paulina y Avelino
Rancho Grande
Quichaura
La Paulina
Mallin Nevado
Laguna del Toro
Don Carlos Invernada
Durruti
Don Carlos Veranada